In [2]:
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

In [15]:
spark = SparkSession.builder.getOrCreate()

22/12/13 03:15:38 WARN Utils: Your hostname, codespaces-fdee62 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/12/13 03:15:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/13 03:15:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
data = pd.read_csv("clean_df.csv")

In [11]:
data["genres_list"]= data["genres_list"].str.replace('\[\]\'', '', regex=True)
# data["genres_list"]= data["genres_list"].str.join(",")
data

,Unnamed: 0,id,title,director,actor,popularity,vote_average,vote_count,runtime,genres_list,key,year
0,0,19995,Avatar,['James Cameron'],"['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",150.437577,7.2,11800,162.0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['culture clash', 'future', 'space war', 'spac...",2009.0
1,1,285,Pirates of the Caribbean: At World's End,['Gore Verbinski'],"['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",139.082615,6.9,4500,169.0,"['Adventure', 'Fantasy', 'Action']","['ocean', 'drug abuse', 'exotic island', 'east...",2007.0
2,2,206647,Spectre,['Sam Mendes'],"['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",107.376788,6.3,4466,148.0,"['Action', 'Adventure', 'Crime']","['spy', 'based on novel', 'secret agent', 'seq...",2015.0
3,3,49026,The Dark Knight Rises,['Christopher Nolan'],"['Christian Bale', 'Michael Caine', 'Gary Oldm...",112.312950,7.6,9106,165.0,"['Action', 'Crime', 'Drama', 'Thriller']","['dc comics', 'crime fighter', 'terrorist', 's...",2012.0
4,4,49529,John Carter,['Andrew Stanton'],"['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...",43.926995,6.1,2124,132.0,"['Action', 'Adventure', 'Science Fiction']","['based on novel', 'mars', 'medallion', 'space...",2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,9367,El Mariachi,['Robert Rodriguez'],"['Carlos Gallardo', 'Jaime de Hoyos', 'Peter M...",14.269792,6.6,238,81.0,"['Action', 'Crime', 'Thriller']","['united states–mexico barrier', 'legs', 'arms...",1992.0
4799,4799,72766,Newlyweds,['Edward Burns'],"['Edward Burns', 'Kerry Bishé', 'Marsha Dietle...",0.642552,5.9,5,85.0,"['Comedy', 'Romance']",[],2011.0
4800,4800,231617,"Signed, Sealed, Delivered",['Scott Smith'],"['Eric Mabius', 'Kristin Booth', 'Crystal Lowe...",1.444476,7.0,6,120.0,"['Comedy', 'Drama', 'Romance', 'TV Movie']","['date', 'love at first sight', 'narration', '...",2013.0
4801,4801,126186,Shanghai Calling,['Daniel Hsia'],"['Daniel Henney', 'Eliza Coupe', 'Bill Paxton'...",0.857008,5.7,7,98.0,[],[],2012.0


In [20]:
data

,Title,Genre,Description,Director,Actors,Year,Rating,Votes
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,8.1,757074
1,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,7.0,485820
2,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,7.3,157606
3,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,7.2,60545
4,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,6.2,393727
...,...,...,...,...,...,...,...,...
995,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,6.2,27585
996,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,5.5,73152
997,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,6.2,70699
998,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,5.6,4881


In [21]:
data_spark=spark.createDataFrame(data)
data_spark.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Votes: long (nullable = true)



/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [22]:
data_spark.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----+------+------+
|               Title|               Genre|         Description|            Director|              Actors|Year|Rating| Votes|
+--------------------+--------------------+--------------------+--------------------+--------------------+----+------+------+
|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|2014|   8.1|757074|
|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|2012|   7.0|485820|
|               Split|     Horror,Thriller|Three girls are k...|  M. Night Shyamalan|James McAvoy, Any...|2016|   7.3|157606|
|                Sing|Animation,Comedy,...|In a city of huma...|Christophe Lourdelet|Matthew McConaugh...|2016|   7.2| 60545|
|       Suicide Squad|Action,Adventure,...|A secret governme...|          David Ayer|Will Smith, Jared...|2016|   6.2|

In [24]:
from pyspark.sql.functions import col,isnan, when, count
data_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_spark.columns]
   ).show()

# df.na.drop().select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
#    ).show()
# df = df.na.drop()


+-----+-----+-----------+--------+------+----+------+-----+
|Title|Genre|Description|Director|Actors|Year|Rating|Votes|
+-----+-----+-----------+--------+------+----+------+-----+
|    0|    0|          0|       0|     0|   0|     0|    0|
+-----+-----+-----------+--------+------+----+------+-----+



22/12/13 03:35:07 WARN TransportChannelHandler: Exception in connection from /172.16.5.4:54504
java.lang.IllegalArgumentException: Too large frame: 1586112597118222328
	at org.sparkproject.guava.base.Preconditions.checkArgument(Preconditions.java:119)
	at org.apache.spark.network.util.TransportFrameDecoder.decodeNext(TransportFrameDecoder.java:148)
	at org.apache.spark.network.util.TransportFrameDecoder.channelRead(TransportFrameDecoder.java:98)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:365)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelRead(AbstractChannelHandlerContext.java:357)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.channelRead(DefaultChannelPipeline.java:1410)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)
	at io.netty.c

Traceback (most recent call last):
  File "/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 62, in worker
  File "/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 556, in loads
    length = read_int(stream)
  File "/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/workspaces/IDS706-Final-Project/env/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 62, i

22/12/13 03:38:32 WARN TransportChannelHandler: Exception in connection from /172.16.5.4:38444
java.lang.IllegalArgumentException: Too large frame: 1586112597118222328
	at org.sparkproject.guava.base.Preconditions.checkArgument(Preconditions.java:119)
	at org.apache.spark.network.util.TransportFrameDecoder.decodeNext(TransportFrameDecoder.java:148)
	at org.apache.spark.network.util.TransportFrameDecoder.channelRead(TransportFrameDecoder.java:98)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:365)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelRead(AbstractChannelHandlerContext.java:357)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.channelRead(DefaultChannelPipeline.java:1410)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)
	at io.netty.c

In [ ]:
from pyspark.ml.feature import Word2Vec
data_spark = data_spark.withColumn("text_splitted", split(lower(col("Description")), " "))

In [ ]:
word2Vec = Word2Vec(vectorSize=100, minCount=0, maxIter=100, inputCol="text_splitted", outputCol="features")
model = word2Vec.fit(documents)